### Get results from Wandb

In [1]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("draftrec/atari_pretrain")

summary_list, config_list, id_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    id_list.append(run.id)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "id": id_list,
    })

runs_df.to_csv("project.csv")

### Read runs

In [2]:
data_ = pd.read_csv('project.csv', index_col=0)
data_

,summary,config,id
0,{'_wandb': {'runtime': 1}},"{'exp_name': 'trajformer_demon_cont_lmbda09', ...",37tnj2e1
1,"{'mean_traj_game_scores': 1, 'act_acc': 0.0662...","{'env': {'game': 'Jamesbond', 'type': 'atari',...",32sdvy1x
2,"{'epoch': 0, 'act_loss': 2.187812092781067, 'r...","{'env': {'game': 'Hero', 'type': 'atari', 'fra...",7sh3biv4
3,"{'max_grad_norm': 1.4167770338058472, 'best_me...","{'env': {'game': 'Frostbite', 'type': 'atari',...",1ieqq4wd
4,"{'min_grad_norm': 8.602183801720286e-09, '_ste...","{'env': {'game': 'Gopher', 'type': 'atari', 'f...",3qc1xzql
...,...,...,...
1115,"{'_step': 117001, '_wandb': {'runtime': 20220}...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",5zlpym7v
1116,"{'_step': 117001, '_wandb': {'runtime': 18252}...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",3oh4f4sw
1117,"{'positive_sim': 0.9868739223480224, 'loss': -...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",385p3sw7
1118,"{'pos_neg_diff': 0.810436166524887, 'positive_...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",njf5gwab


### Filter based on conditions

In [3]:
group_name = 'baseline'
exp_name = 'trajformer_demon_cont_lmbda09'
model_path = '0/10/model.pth'

In [4]:
configs = data_['config']
indexs = []
for idx, config in enumerate(configs):
    cfg = eval(config)
    if len(cfg) == 0:
        continue
        
    run_exp_name = cfg['exp_name']
    run_group_name = cfg['group_name']

    # condition
    if run_exp_name == exp_name and run_group_name == group_name:
        if 'env' in cfg:
            indexs.append(idx)

In [5]:
data = data_.iloc[indexs]
envs = []
for config in data['config']:
    envs.append(eval(config)['env']['game'])
data['env'] = envs
print(len(data))
data

26


/tmp/ipykernel_164559/3806693401.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['env'] = envs


,summary,config,id,env
9,"{'_wandb': {'runtime': 51857}, 'act_f1': 0.217...","{'env': {'game': 'UpNDown', 'type': 'atari', '...",1bckj5uc,UpNDown
10,"{'pos_neg_diff': 0.6245496030745468, 'loss': 1...","{'env': {'game': 'Seaquest', 'type': 'atari', ...",23i9avnz,Seaquest
32,"{'max_grad_norm': 0.5404402639686576, 'best_me...","{'env': {'game': 'Qbert', 'type': 'atari', 'fr...",1itelag0,Qbert
33,"{'idm_acc': 0.19338453498474628, '_runtime': 5...","{'env': {'game': 'Pong', 'type': 'atari', 'fra...",1j7to096,Pong
34,"{'_wandb': {'runtime': 52044}, 'rew_loss': 0, ...","{'env': {'game': 'RoadRunner', 'type': 'atari'...",1p38yv8c,RoadRunner
35,"{'loss': 1.349123662080222, 'obs_loss': 1.3529...","{'env': {'game': 'PrivateEye', 'type': 'atari'...",376gpxkp,PrivateEye
48,"{'act_f1': 0.25158480040333964, 'act_acc': 0.6...","{'env': {'game': 'Kangaroo', 'type': 'atari', ...",2v1hi0uz,Kangaroo
49,"{'_timestamp': 1667810245, 'act_acc': 0.657432...","{'env': {'game': 'MsPacman', 'type': 'atari', ...",65mdd95b,MsPacman
50,"{'obs_loss': 1.3832031036134012, 'pos_neg_diff...","{'env': {'game': 'Krull', 'type': 'atari', 'fr...",11wpwe2z,Krull
51,"{'neg_sim': 0.003286767248720496, 'pos_sim': 0...","{'env': {'game': 'KungFuMaster', 'type': 'atar...",362bgmrw,KungFuMaster


### 1. Restore Saved Models

In [6]:
import pathlib
base_path = str(pathlib.Path().resolve())

artifact_dict = {}
for run_id, env in zip(data['id'], data['env']):
    print(env, run_id)
    try:
        name = env + '/' + model_path 
        path = base_path + '/' + name
        wandb.restore(name, run_path="draftrec/atari_pretrain/" + run_id)
        artifact_dict[path] = name 
    except:
        continue

UpNDown 1bckj5uc
Seaquest 23i9avnz
Qbert 1itelag0
Pong 1j7to096
RoadRunner 1p38yv8c
PrivateEye 376gpxkp
Kangaroo 2v1hi0uz
MsPacman 65mdd95b
Krull 11wpwe2z
KungFuMaster 362bgmrw
Frostbite 2gm4iv4f
Gopher 125hf5rz
Jamesbond 1a4vebqd
Hero q3nwbyqx
ChopperCommand 1s5fes6j
DemonAttack 1bzzgspc
CrazyClimber 1m748fxo
Freeway 2vyt5rkb
Boxing 1r9w1t0n
BattleZone 84xqan1z
BankHeist 1kv2lyiw
Breakout 3nbpgfb5
Asterix h3mpgf56
Amidar 2rq62one
Alien 9l37in8j
Assault h1klwvb3


### Save to artifact

In [7]:
wandb.init(project='atari_pretrain',
           entity='draftrec',
           group=exp_name,
           settings=wandb.Settings(start_method="thread"))  
artifact = wandb.Artifact(name=exp_name, type='model')

# save models
for path, name in artifact_dict.items():
    artifact.add_file(path, name=name)

wandb.run.finish_artifact(artifact)

wandb: Currently logged in as: joonleesky (draftrec). Use `wandb login --relogin` to force relogin
/opt/conda/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


### Remove artifact

In [8]:
import shutil
for env in data['env']:
    shutil.rmtree('./'+env, ignore_errors=True)

### 2. Generate json file

In [ ]:
def get_results_dict(data):
    results = []
    for idx in range(len(data)):
        row = data.iloc[idx]
        summary = eval(row['summary'])
        config = eval(row['config'])

        if 'env' not in config:
            continue

        game = config['env']['game']
        try:
            import pdb
            pdb.set_trace()
            score = summary['eval_mean_traj_game_scores']
        except:
            continue
        results.append([0, game, score, 0])

    return results
    
results = get_results_dict(data)

> /tmp/ipykernel_10477/287021157.py(15)get_results_dict()
     13             import pdb
     14             pdb.set_trace()
---> 15             score = summary['eval_mean_traj_game_scores']
     16         except:
     17             continue



ipdb>  summary


{'epoch': 10, 'act_f1': 0.3645197793617396, 'reward_f1': 0.1449814126394052, 'max_grad_norm': 1.2048966879767131, 'min_grad_norm': 4.187063070845566e-09, 'mean_grad_norm': 0.09672310950391017, 'loss': 1.4424609811648443, 'neg_sim': 0.0034324854584025497, 'pos_sim': 0.7590320446303866, 'reward_ratio': 0.06420321016050802, '_step': 234370, '_timestamp': 1666850956, 'best_metric_val': 0.3689052111797868, '_wandb': {'runtime': 77521}, '_runtime': 77521, 'pos_neg_diff': 0.7555995596456657}


ipdb>  q


> /tmp/ipykernel_10477/287021157.py(14)get_results_dict()
     12         try:
     13             import pdb
---> 14             pdb.set_trace()
     15             score = summary['eval_mean_traj_game_scores']
     16         except:



ipdb>  q


> /tmp/ipykernel_10477/287021157.py(15)get_results_dict()
     13             import pdb
     14             pdb.set_trace()
---> 15             score = summary['eval_mean_traj_game_scores']
     16         except:
     17             continue



In [8]:
data

,summary,config,id,env
66,"{'epoch': 10, 'act_f1': 0.3645197793617396, 'r...","{'env': {'game': 'UpNDown', 'type': 'atari', '...",1ewu8l5y,UpNDown
67,"{'min_grad_norm': 1.885691627837832e-09, 'best...","{'env': {'game': 'Seaquest', 'type': 'atari', ...",2idssz6q,Seaquest
74,"{'_timestamp': 1666772788, 'reward_ratio': 0.0...","{'env': {'game': 'PrivateEye', 'type': 'atari'...",5y9cakz7,PrivateEye
75,"{'act_f1': 0.40050761492385406, '_runtime': 77...","{'env': {'game': 'Qbert', 'type': 'atari', 'fr...",31zhorvk,Qbert
76,"{'_wandb': {'runtime': 78307}, 'act_f1': 0.377...","{'env': {'game': 'RoadRunner', 'type': 'atari'...",i7qviowp,RoadRunner
77,"{'max_grad_norm': 0.5247982027246377, 'reward_...","{'env': {'game': 'Pong', 'type': 'atari', 'fra...",2hh3412n,Pong
145,"{'mean_grad_norm': 0.07623743895835024, '_step...","{'env': {'game': 'Krull', 'type': 'atari', 'fr...",1jmknt49,Krull
146,"{'epoch': 10, 'neg_sim': -0.002199352687703164...","{'env': {'game': 'KungFuMaster', 'type': 'atar...",1txoflsv,KungFuMaster
147,"{'loss': 0.4945657217728736, '_wandb': {'runti...","{'env': {'game': 'Kangaroo', 'type': 'atari', ...",12sqezq0,Kangaroo
148,"{'_wandb': {'runtime': 49101}, '_timestamp': 1...","{'env': {'game': 'MsPacman', 'type': 'atari', ...",3g4050i5,MsPacman


In [18]:
import json

def generate_json_file(file_name, results):
    data = {}
    data['data'] = results
    path = file_name + '.json'
    with open(path, 'w') as json_file:
        json.dump(data, json_file)

In [19]:
generate_json_file('byol_impala', results)